In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [6]:
df=pd.read_csv(r"D:\Big Data Analytics- Sem 3\Capstone Project\Toronto\Milestone1&2\listing_clean_1.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21114 entries, 0 to 21113
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              21114 non-null  int64  
 1   source                          21114 non-null  object 
 2   description                     20560 non-null  object 
 3   neighborhood_overview           11111 non-null  object 
 4   host_id                         21114 non-null  int64  
 5   host_location                   15647 non-null  object 
 6   host_about                      9846 non-null   object 
 7   host_response_time              15043 non-null  object 
 8   host_response_rate              15043 non-null  object 
 9   host_acceptance_rate            15765 non-null  object 
 10  host_is_superhost               20740 non-null  object 
 11  host_neighbourhood              8464 non-null   object 
 12  host_verifications              

In [8]:
# Define the mapping functions
df['host_response_time'] = df['host_response_time'].replace(
  [np.nan, 'within an hour', 'within a few hours', 'within a day', 'a few days or more'],
  [0, 1, 2, 3, 4])

def map_response_rate(rate):
    if pd.isna(rate):
        return 0
    rate = int(str(rate).strip('%'))
    if 90 <= rate <= 100:
        return 4
    elif 70 <= rate < 90:
        return 3
    elif 50 <= rate < 70:
        return 2
    elif 0 <= rate < 50:
        return 1
    else:
        return 0

def map_acceptance_rate(rate):
    if pd.isna(rate):
        return 0
    rate = int(str(rate).strip('%'))
    if 90 <= rate <= 100:
        return 4
    elif 70 <= rate < 90:
        return 3
    elif 50 <= rate < 70:
        return 2
    elif 0 <= rate < 50:
        return 1
    else:
        return 0

# Apply the mapping functions to the DataFrame

df['host_response_rate'] = df['host_response_rate'].apply(map_response_rate)
df['host_acceptance_rate'] = df['host_acceptance_rate'].apply(map_acceptance_rate)




In [9]:
pd.set_option('display.max_rows', None)
index = df[['host_response_time', 'host_response_rate', 'host_acceptance_rate']]
for i in index:
    print(df[i].value_counts())
    print("-------------------------------")

host_response_time
1    10831
0     6071
2     2696
3     1066
4      450
Name: count, dtype: int64
-------------------------------
host_response_rate
4    13390
0     6071
3      988
1      450
2      215
Name: count, dtype: int64
-------------------------------
host_acceptance_rate
4    9920
0    5349
3    3137
2    1461
1    1247
Name: count, dtype: int64
-------------------------------


In [10]:
# Adding two columns to create the new column
df['host_response_score'] = df['host_response_time'] + df['host_response_rate'] + df['host_acceptance_rate']

# Dropping the two columns used for the addition
df = df.drop(columns=['host_response_time', 'host_response_rate','host_acceptance_rate' ])

In [11]:
df['host_response_score'].value_counts()

host_response_score
9     8762
0     4871
8     2712
7     1338
10    1069
6      665
4      655
5      297
1      245
11     194
3      189
2      117
Name: count, dtype: int64

In [12]:
def within_canada(location):
    if 'Canada' in str(location):
        return 1
    else:
        return 0

# Assuming df is your DataFrame and 'host_location' is the column you want to check
df['host_location'] = df['host_location'].apply(within_canada)

Alternate function to do the within_canada task     
df['host_location'] = df['host_location'].str.contains('Canada', case=False, na=False).astype(int)

In [13]:
#Changes t to 1 and f to 0 in host_is_superhost column
df['host_is_superhost'] = df['host_is_superhost'].str.contains('t', na=False).astype(int)

In [14]:
#Dropping host_neighborhood and neighborhood columns, and keeping only neighborhood_cleansed column.

df= df.drop(columns=['neighbourhood', 'host_neighbourhood'], axis=1)

In [15]:
#Dropping host_verifications column because it has not much importance in our predictions
df= df.drop(columns=['host_verifications'], axis=1)

In [16]:
#Changes t to 1 and f to 0 in host_has_profile_pic and host_identity_verified columns
df['host_has_profile_pic'] = df['host_has_profile_pic'].str.contains('t', na=False).astype(int)
df['host_identity_verified'] = df['host_identity_verified'].str.contains('t', na=False).astype(int)

In [17]:
# Adding two columns to create the new column
df['host_score'] = df['host_location'] + df['host_is_superhost'] + df['host_has_profile_pic'] + df['host_identity_verified']

# Dropping the two columns used for the addition
df = df.drop(columns=['host_location','host_is_superhost', 'host_has_profile_pic','host_identity_verified' ])

In [18]:
corr_matrix = df[['bathrooms', 'beds', 'bedrooms', 'accommodates']].corr()

# Print the correlation matrix
print(corr_matrix)

              bathrooms      beds  bedrooms  accommodates
bathrooms      1.000000  0.424920  0.659930      0.580114
beds           0.424920  1.000000  0.601887      0.673872
bedrooms       0.659930  0.601887  1.000000      0.708647
accommodates   0.580114  0.673872  0.708647      1.000000


property_type room_type accommodates bathrooms bathrooms_text bedrooms beds     
If the model's accuracy score is low and any of these columns has high feature importance, we will come back and revisit the choice of columns, feature engg feature selection and encoding to further improve the model metrices.

In [19]:
df = df.drop(['room_type', 'bathrooms', 'bedrooms', 'beds', 'source'], axis=1)

In [20]:
df['property'] = df['property_type'].astype('category').cat.codes
df['neighbourhood'] = df['neighbourhood_cleansed'].astype('category').cat.codes
df['bathrooms'] = df['bathrooms_text'].astype('category').cat.codes

In [21]:
df = df.drop(['property_type', 'neighbourhood_cleansed', 'bathrooms_text'], axis=1)

We believe that the columns number_of_reviews_l30d, number_of_reviews_ltm, reviews_per_month information overlaps with the number_of_reviews column so we will be dropping but if needed we will revisit to check if they can be used to improve the metrices.

The columns first_reviews and last_review aren't that important to predict price but revisit if needed.

In [22]:
df = df.drop(['first_review', 'last_review', 'calendar_last_scraped', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'license','reviews_per_month'], axis=1)

In [23]:
df['count_amenities'] = df['amenities'].apply(len)
df['count_amenities'] = df['count_amenities'].astype(int)
		

In [24]:
# create a dictionary to store the count of each item
item_counts = {}

# iterate over each row in the 'amenities' column
for amenities in df['amenities']:
    # split the amenities string into individual items
    items = amenities.split(', ')
    # iterate over each item
    for item in items:
        # strip leading/trailing whitespace
        item = item.strip()
        # increment the count for this item
        if item in item_counts:
            item_counts[item] += 1
        else:
            item_counts[item] = 1

# sort the item counts in descending order
sorted_item_counts = sorted(item_counts.items(), key=lambda x: x[1], reverse=True)

# print the sorted item counts
for item, count in sorted_item_counts:
    print(f"{item}: {count}")


"Smoke alarm": 18940
"Kitchen": 18926
"Wifi": 18432
"Essentials": 17272
"Hangers": 16490
"Hot water": 15530
"Hair dryer": 15229
"Dishes and silverware": 15052
"Cooking basics": 13895
"Refrigerator": 13787
"Iron": 13702
"Carbon monoxide alarm": 13620
"Microwave": 13522
"Air conditioning": 13505
"Shampoo": 13431
"Bed linens": 13152
"Fire extinguisher": 12139
"Washer": 12015
"Heating": 11422
"Long term stays allowed": 10466
"Dedicated workspace": 10314
"Hot water kettle": 10231
"Self check-in": 10152
"Cleaning products": 9834
"First aid kit": 9284
"Dishwasher": 9005
"Extra pillows and blankets": 8880
"Freezer": 8721
"Oven": 8662
"Toaster": 8612
"Dryer": 7866
"Shower gel": 7772
"Free parking on premises": 7736
"Bathtub": 7713
"TV"]: 7694
"Private entrance": 7690
"Wine glasses": 7610
"Body soap": 7414
"Dining table": 7268
"Coffee maker": 6797
"Conditioner": 6655
"Central heating": 6618
"Room-darkening shades": 6512
"Coffee": 6287
"Elevator": 6021
"Exterior security cameras on property": 535

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21114 entries, 0 to 21113
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              21114 non-null  int64  
 1   description                     20560 non-null  object 
 2   neighborhood_overview           11111 non-null  object 
 3   host_id                         21114 non-null  int64  
 4   host_about                      9846 non-null   object 
 5   accommodates                    21114 non-null  int64  
 6   amenities                       21114 non-null  object 
 7   price                           15903 non-null  object 
 8   minimum_nights                  21114 non-null  int64  
 9   maximum_nights                  21114 non-null  int64  
 10  minimum_nights_avg_ntm          21114 non-null  float64
 11  maximum_nights_avg_ntm          21114 non-null  float64
 12  has_availability                

In [26]:
df1=df.drop(['id', 'host_id', 'description', 'neighborhood_overview', 'host_about', 'amenities'], axis=1)
index = df1.columns
for i in index:
    print(df1[i].value_counts())
    print("-------------------------------")
    

accommodates
2     8377
4     3958
1     2919
3     2150
6     1514
5     1114
8      488
7      246
10     143
12      76
9       54
16      35
14      19
11      15
13       4
15       2
Name: count, dtype: int64
-------------------------------
price
$100.00       297
$150.00       278
$200.00       277
$120.00       269
$80.00        245
$60.00        212
$65.00        208
$90.00        206
$55.00        201
$70.00        196
$50.00        187
$250.00       182
$99.00        177
$300.00       176
$110.00       161
$75.00        160
$40.00        152
$160.00       152
$140.00       150
$130.00       138
$85.00        137
$129.00       134
$95.00        125
$45.00        125
$180.00       124
$115.00       122
$125.00       122
$79.00        108
$149.00       102
$89.00        100
$69.00         98
$48.00         94
$350.00        92
$175.00        91
$135.00        90
$199.00        89
$105.00        86
$98.00         84
$119.00        81
$145.00        79
$68.00         77
$56.00   

In [27]:
desired_location = "D:\Big Data Analytics- Sem 3\Capstone Project\Toronto\Milestone1&2\listing_clean_2.csv"

# Save the DataFrame as a CSV file
df.to_csv(desired_location, index=False)